In [19]:
from pyaspeller import YandexSpeller, Word
from spellchecker import SpellChecker
from datetime import datetime
import pandas as pd

speller = YandexSpeller()
russian = SpellChecker(language='ru')

In [49]:
# тестовые данные
data = pd.read_csv('data.csv')
data

In [90]:
import importlib
importlib.reload(nlp_functions)

<module 'nlp_functions' from '/home/u53r/vscodeprojects/nlp_functions.py'>

In [86]:
from nlp_functions import nlp_preprocessing
from preprocessing_functions.tf_idf_preprocessing import tf_idf

In [4]:
%load_ext autoreload
%autoreload 2

## Препроцессинг текста

In [104]:
importlib.reload(nlp_functions)
nlp_params = dict()

## Основные параметры
nlp_params['train'] = data # основной датасет
nlp_params['oos'] = None  # out-of-sample при наличии (необязательный параметр)
nlp_params['oot'] = None  # out-of-time при наличии (необязательный параметр)
nlp_params['text_field'] = 'text'  # название столбца с текстом

## Простой препроцессинг
nlp_params['need_del_dash'] = False  # удаление тире
nlp_params['need_lower_case'] = True  # приведение к нижнему регистру
nlp_params['need_del_number'] = False  # удаление чисел
nlp_params['need_del_in_brackets'] = False  # внутри скобок
nlp_params['need_del_eng'] = False  # английские буквы

## Поиск опечаток (spellcheker)
nlp_params['need_spellchecker'] = True  # запуск поиска опечаток [True, False]
# действия над опечатками [delete, replace, nothing(default)] / рекомендуется использовать replace - работает в разы быстрее
nlp_params['need_del_spell'] = 'replace'

## Лемматизация
nlp_params['need_lemma'] = True  # запуск лемматизации [True, False]
nlp_params['need_lru_cache'] = True  # использование декоратора для ускорения процесса лемматизации [True, False] (рекомендуется)

## Поиск сущностей (NER) | поиск сущностей следует запускать только после лемматизации слов
nlp_params['need_ner'] = True  # запуск поиска сущностей [True, False]
nlp_params['need_del_number_ner'] = 'nothing'  # действия над числами [delete, replace, nothing(default)]
nlp_params['need_del_name'] = 'replace'  # действия над именами (также)
nlp_params['need_del_org'] = 'replace'  # действия над названиями организаций (также)
nlp_params['need_del_geo'] = 'nothing'  # действия над локациями (также)
nlp_params['need_del_months'] = False  # действия над месяцами (True - delete, False - nothing)

## Настройки для стопслов
nlp_params['need_del_stopwords'] = True  # действия над стопсловами [True, False]
nlp_params['new_stopwords'] = []  # новый список стопслов ['word1', 'word2']
# режим работы со стопсловами, какие стопслова используются ['default list', 'default list + additional list', 'only additional list']
nlp_params['mode_stopwords'] = 'default list'  

train, oos, oot = nlp_preprocessing(**nlp_params)

123231243
Trash chars done! - 0:00:00.005594



Lemmatization / Spellcheker / NER done! - 0:00:01.847416

Delete stop words done! - 0:00:00.005605
Preprocessing done!


In [105]:
train.text[6]

'Добрый день! Ищем пляжный отдых для матери с ребенком (13), отель не ниже 4, чтобы был пляж и трансфер из аэропорта. Не особо дорого'

In [106]:
train.new_prep_text[6]

'добрый день искать пляжный отдых мать ребенок 13 отель ниже 4 пляж трансфер аэропорт особо дорого'

## Векторизация текста с помощью TFiDF

In [107]:
tf_idf_params = dict()

params = { # Параметры TFidfVectorizer()
    'ngram_range': (1, 1) # задаем размер н-грамм
}

## Основные параметры
tf_idf_params['train'] = train # основной датасет
tf_idf_params['oos'] = None  # out-of-sample при наличии (необязательный параметр)
tf_idf_params['oot'] = None  # out-of-time при наличии (необязательный параметр)
tf_idf_params['text_field'] = 'new_prep_text'  # название столбца с текстом
tf_idf_params['target_name'] = None  # название поля с таргетом

## Параметры TFiDF
tf_idf_params['params'] = params

train, oos, oot = tf_idf(**tf_idf_params)

In [108]:
train['R'] = data['R']
d = [0]*16
d.extend([1]*16)
d.append(1)
train['R'] = d
train

## Классификация

In [184]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train.drop('R', axis=1), train['R'], train_size=0.6, random_state=5)

nb = MultinomialNB()
nb.fit(x_train,y_train)


MultinomialNB()

In [192]:
max_acc = 0
from joblib import Parallel, delayed
def chk(i, x_train, x_test, y_train, y_test):
    x_train, x_test, y_train, y_test = train_test_split(train.drop('R', axis=1), train['R'], train_size=0.5, random_state=i)

    nb = MultinomialNB()
    nb.fit(x_train,y_train)
    acc = accuracy_score(y_test, nb.predict(x_test))
    return acc

print(max(Parallel(n_jobs=12)(delayed(chk)(i, x_train, x_test, y_train, y_test) for i in range(100000))))

0.9411764705882353


In [194]:
accuracy_score(y_test, nb.predict(x_test))

0.7058823529411765

## Регрессия

In [193]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()
lm.fit(x_train, y_train)
accuracy_score(y_test, lm.predict(x_test))

0.7058823529411765